In [1]:
import geopandas as gpd 
from shapely.geometry import Point
from assign_taxi_zone import assign_taxi_zones
import pandas as pd
import dask.dataframe as dd 
import numpy as np
import os 
from dask.diagnostics import ProgressBar
from glob import glob
import pickle
from tqdm import tqdm
from random import shuffle
from datetime import datetime
from itertools import product

In [2]:
pbar=ProgressBar()
pbar.register()

In [3]:
data_path='./data_2016/'
data_type='*.csv'

In [4]:
 #폴더에 있는 파일 list
 data_list = sorted(glob(os.path.join(data_path, data_type)))

In [ ]:
data_list

In [6]:
for i, data in enumerate(data_list):

         tmp_str=data.replㅇace('.csv','')
         month=tmp_str.replace('./data_2016\\cleansing_newyork_yellow_taxi_','')
         

         print(month)

         trip_csv=dd.read_csv(data)
         trip_csv['PULocationID']=trip_csv.map_partitions(assign_taxi_zones,'pickup_longitude', 'pickup_latitude',meta=('PULocationID',int))
         trip_csv['DOLocationID']=trip_csv.map_partitions(assign_taxi_zones,'dropoff_longitude', 'dropoff_latitude',meta=('DOLocationID',int))

         trip_csv.to_parquet(f'./data_2016_location/trips_{month}.parquet',has_nulls=True,object_encoding='json',compression='SNAPPY')

2016-02
[########################################] | 100% Completed | 37min 42.8s


In [5]:
trip_csv=dd.read_csv('./data_2016/cleansing_newyork_yellow_taxi_2016_03.csv')

In [6]:
trip_csv.head()

[########################################] | 100% Completed |  0.7s


,Unnamed: 0,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,total_amount,trip_duration,pickup_time,temp_pickup_time,speed,pickup_cluster,time_bin,pickup_date,pickup_time_hhmmss,pickup_weekday_index,pickup_holiday,pickup_time_index
0,0,2,0.70,-73.947250,40.763771,-73.992012,40.735390,5.8,3.00,1.456392e+09,2016-02-25 17:24:20,14.000000,18,60536,2016-02-25,17:24:20,4,0,34
1,1,2,5.52,-73.983017,40.750992,-73.988586,40.758839,21.3,21.00,1.456413e+09,2016-02-25 23:10:50,15.771429,9,60571,2016-02-25,23:10:50,4,0,46
2,2,6,1.99,-73.992340,40.758202,-73.964355,40.757977,11.5,10.85,1.454256e+09,2016-02-01 00:00:01,11.004608,5,56976,2016-02-01,00:00:01,1,0,0
3,3,1,1.50,-73.981453,40.749722,-73.982323,40.763985,7.8,5.20,1.454256e+09,2016-02-01 00:00:04,17.307692,9,56976,2016-02-01,00:00:04,1,0,0
4,4,1,5.60,-74.000603,40.729755,-73.951324,40.669834,25.3,20.90,1.454256e+09,2016-02-01 00:00:05,16.076555,7,56976,2016-02-01,00:00:05,1,0,0


In [ ]:
trip_csv['PULocationID']=trip_csv.map_partitions(assign_taxi_zones,'pickup_longitude', 'pickup_latitude',meta=('PULocationID',int))

In [ ]:
trip_csv['DOLocationID']=trip_csv.map_partitions(assign_taxi_zones,'dropoff_longitude', 'dropoff_latitude',meta=('DOLocationID',int))


In [ ]:
test=dd.read_parquet('./data_2016_location/trips_2016_01.parquet',columns=['PULocationID','DOLocationID'])

In [3]:
data_path='./data_2016_location/'
data_type='*.parquet'

In [4]:
data_list = sorted(glob(os.path.join(data_path,data_type)))
data_list

['./data_2016_location\\trips_2016-02.parquet']

In [7]:
dat1 = pd.read_parquet('./data_2016_location\\trips_2016_03.parquet')

In [8]:
dat1.head()

,Unnamed: 0,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,total_amount,trip_duration,pickup_time,...,speed,pickup_cluster,time_bin,pickup_date,pickup_time_hhmmss,pickup_weekday_index,pickup_holiday,pickup_time_index,PULocationID,DOLocationID
__null_dask_index__,,,,,,,,,,,,,,,,,,,,,
0,0,2,0.70,-73.947250,40.763771,-73.992012,40.735390,5.8,3.00,1.456392e+09,...,14.000000,18,60536,"b'""2016-02-25""'","b'""17:24:20""'",4,0,34,202,234
1,1,2,5.52,-73.983017,40.750992,-73.988586,40.758839,21.3,21.00,1.456413e+09,...,15.771429,9,60571,"b'""2016-02-25""'","b'""23:10:50""'",4,0,46,164,230
2,2,6,1.99,-73.992340,40.758202,-73.964355,40.757977,11.5,10.85,1.454256e+09,...,11.004608,5,56976,"b'""2016-02-01""'","b'""00:00:01""'",1,0,0,48,229
3,3,1,1.50,-73.981453,40.749722,-73.982323,40.763985,7.8,5.20,1.454256e+09,...,17.307692,9,56976,"b'""2016-02-01""'","b'""00:00:04""'",1,0,0,170,163
4,4,1,5.60,-74.000603,40.729755,-73.951324,40.669834,25.3,20.90,1.454256e+09,...,16.076555,7,56976,"b'""2016-02-01""'","b'""00:00:05""'",1,0,0,114,62


In [5]:
for i, data in enumerate(data_list):

        #Read Parquet and Convert to DataFrame 
      
     
        dat1 = pd.read_parquet(data)

        decoded_date=dat1['pickup_date'].str.decode(encoding='UTF-8')

        striped_date=decoded_date.str.strip('"')
        
        dat1['pickup_date']=striped_date
        # Month setting
        month = i + 1
        print(data)
        #print(f'{month} month start...')

 
        # Pre-process other month
        if month <= 9:
            year_month = f'2016-0{month}'
        else:
            year_month = f'2016-{month}'
            
        # Unique list setting
        date_list = [x for x in sorted(list(set(dat1['pickup_date']))) if x[:7] == year_month]
        hour_list = range(0,48)
        location_list = list(set(dat1['PULocationID']))

        # Make processed list
        location_list2, date_list2, hour_list2, weekday_list = list(), list(), list(), list()

        for location, date, hour in product(location_list, date_list, hour_list):
            location_list2.append(location)
            date_list2.append(date)
            hour_list2.append(hour)
            weekday_list.append(datetime.strptime(date, '%Y-%m-%d').weekday())

        # Count
        count_list = list()

        for i in tqdm(range(len(location_list2))):
            location_dat = dat1[dat1['PULocationID'] == location_list2[i]]
            date_dat = location_dat[location_dat['pickup_date'] == date_list2[i]]
            hour_dat = date_dat[date_dat['pickup_time_index'] == hour_list2[i]]
            count_list.append(len(hour_dat))
    
        # Total_data make & save
        total_dat = pd.DataFrame({
            'location': location_list2,
            'date': date_list2,
            'weekday': weekday_list,
            'hour': hour_list2,
            'count': count_list
        })
        total_dat.to_csv(os.path.join('./data_2016_location/', f'newyork_yellow_taxi_2016-0{month}_count.csv'), index=False)


./data_2016_location\trips_2016-02.parquet
100%|██████████| 342432/342432 [3:48:30<00:00, 24.98it/s]
